In [2]:
%pip install torch gym gymnasium matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 11.3 MB/s eta 0:00:0000:0100:01
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827694 sha256=6c15a2e2e32d47f6266a08fcac1419dc91f29d04d3589ff5ffc605c50b3643e7
  Stored in directory: /Users/mcelikik/Library/Caches/pip/wheels/bf/6b/4b/31afd00fd7e6b9af007bcddfe33e2ff1a1235210639e226214
Successfully built gym

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Set device for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
EPISODES = 1000
LEARNING_RATE = 1e-2
GAMMA = 0.99

# Create environment
env = gym.make("CartPole-v1")

# Define the Policy Network (MLP) that outputs action probabilities
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=-1)  # Output probabilities for actions
        )

    def forward(self, x):
        return self.fc(x)

# Initialize policy network and optimizer
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
policy_net = PolicyNetwork(state_dim, action_dim).to(device)
optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)

# Function to compute discounted returns efficiently using vectorized operations
def compute_discounted_returns(rewards, gamma=GAMMA):
    R = 0
    returns = []
    # Compute returns in reverse order
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    return torch.tensor(returns, dtype=torch.float32, device=device)

# Training loop
reward_history = []

for episode in range(EPISODES):
    state, _ = env.reset()
    log_probs, rewards = [], []
    done = False
   
    # Generate an episode
    while not done:
        print(".", end="")
        state_tensor = torch.FloatTensor(state).to(device)
        # Get action probabilities from policy network
        action_probs = policy_net(state_tensor)
        dist = torch.distributions.Categorical(action_probs)
        action = dist.sample()

        # Store log probability for policy gradient update
        log_probs.append(dist.log_prob(action))
        state, reward, done, truncated, _ = env.step(action.item())
        print(reward, end="")
        rewards.append(reward)

    # Compute discounted returns and normalize for stability
    returns = compute_discounted_returns(rewards)
    returns = (returns - returns.mean()) / (returns.std() + 1e-9)

    # Compute policy loss: negative log-likelihood weighted by returns
    policy_loss = -torch.stack(log_probs) * returns
    loss = policy_loss.sum()

    # Update policy network
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_reward = sum(rewards)
    reward_history.append(total_reward)

    if (episode + 1) % 50 == 0:
        print(f"Episode {episode+1}/{EPISODES} - Total Reward: {total_reward:.2f}")

# Plot training performance
plt.plot(reward_history)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("REINFORCE on CartPole-v1")
plt.show()

env.close()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 50/1000 - Total Reward: 79.00
Episode 100/1000 - Total Reward: 173.00


KeyboardInterrupt: 